<a href="https://colab.research.google.com/github/Youssef-Chip/lab1/blob/main/lab2/01_data_and_baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
DATA_DIR = "/Users/mr.youssef/Dropbox/lab1/lab2/dataset/UCI_HAR_Dataset"

In [ ]:
def load_inertial_signals(subset):
    signals = ['body_acc_x', 'body_acc_y', 'body_acc_z', 'body_gyro_x', 'body_gyro_y', 'body_gyro_z','total_acc_x', 'total_acc_y', 'total_acc_z']
    loaded_signals = []
    for sig in signals:
        path = os.path.join(DATA_DIR, subset, 'Inertial Signals', f'{sig}_{subset}.txt')
        loaded_signals.append(pd.read_csv(path, sep=r'\s+', header=None).values)
        # each file has shape: (N, 128) where N is the number of windows (2.56 seconds per window) and 128 is sequential time steps for that specific window
    # Go from 9 arrays of: (N, 128) to 1 matrix (N, 128, 9)
    return np.dstack(loaded_signals)

In [ ]:
def load_y_and_subjects(subset):
    # y ==> 0 to 5 multiclass describing the activity
    # subjecs: each row has the ID of the specific human volunteer who was wearing the phone for that specific window
    y_path = os.path.join(DATA_DIR, subset, f'y_{subset}.txt')
    sub_path = os.path.join(DATA_DIR, subset, f'subject_{subset}.txt')
    y = pd.read_csv(y_path, sep=r'\s+', header=None).values.squeeze() - 1 # 0-indexed
    subjects = pd.read_csv(sub_path, sep=r'\s+', header=None).values.squeeze()
    return y, subjects

In [ ]:
X_train = load_inertial_signals('train')
X_test = load_inertial_signals('test')
y_train, subjects_train = load_y_and_subjects('train')
y_test, subjects_test = load_y_and_subjects('test')

In [ ]:
X_train.shape

(7352, 128, 9)

In [ ]:
X_test.shape

(2947, 128, 9)

In [ ]:
print(y_train.shape, subjects_train.shape)

(7352,) (7352,)


In [ ]:
print(y_test.shape, subjects_test.shape)

(2947,) (2947,)


In [ ]:
print(f"NaNs: {np.isnan(X_train).sum()} and Infs: {np.isinf(X_train).sum()}\n")

NaNs: 0 and Infs: 0



In [ ]:
np.bincount(y_train)

array([1226, 1073,  986, 1286, 1374, 1407])

In [ ]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5])

In [ ]:
np.unique(subjects_train)


array([ 1,  3,  5,  6,  7,  8, 11, 14, 15, 16, 17, 19, 21, 22, 23, 25, 26,
       27, 28, 29, 30])

In [ ]:
# leakage check
np.random.seed(42)
unique_train_subs = np.unique(subjects_train)
# Hold out 4 subjects from the training pool for checking
val_subs = np.random.choice(unique_train_subs, size=4, replace=False)
# get everyone from the training pool instead the 4 subjects above
train_subs = np.setdiff1d(unique_train_subs, val_subs)
# Prove overlap is 0
train_val_overlap = set(train_subs).intersection(set(val_subs))
train_test_overlap = set(train_subs).intersection(set(subjects_test))

In [ ]:
print(len(train_val_overlap), len(train_test_overlap))

0 0


In [16]:
class MajorityClassPredictor:
  """
  Trivial baseline model that always predicts the majority class.
  """
  def __init__(self):
    self.majorityClass = None

  def fit(self, trainLabels):
    """
    Method to find the most frequent class in the training labels
    """
    values, count = np.unique(trainLabels, return_counts=True)
    self.majorityClass = values[np.argmax(count)]

  def predict(self, numSamples):
    """
    Method to blindly return the majority class for every sample.
    """
    return np.full(numSamples, self.majorityClass)

In [ ]:
class BaselineMLP(nn.Module):
  """
  Baseline MLP model for classification tasks.
  - inputDim: number of input features
  - hiddenDim: Our hyperparameter for the hidden layer. This defines the size of
               the hidden layer.
  - numClasses: number of output classes
  """
  def __init__(self, inputDim, hiddenDim, numClasses):
    super(BaselineMLP, self).__init__()

    # Define the layers
    # Layer 1: Input -> Hidden
    self.layer1 = nn.Linear(inputDim, hiddenDim)
    self.activation = nn.ReLU()
    self.drop1 = nn.Dropout(0.3) # turn off 30% of neurons everytime data comes throuhg
    # Layer 2: Hidden -> Output
    self.layer2 = nn.Linear(hiddenDim, numClasses)

  def forward(self, X):
    """
    Method to perform forward propagation from:
    input layer -> hidden layer -> output layer.

    The 2D input X passes through the first layer, goes through ReLU activation,
    introducing non-linearity, and then passes through the second layer.
    """
    X = self.layer1(X)
    X = self.activation(X)
    X = self.drop1(X)
    X = self.layer2(X)
    return X

In [ ]:
def create_dataloader(X, y, batch_size=64, shuffle=False):
    # Flatten: (N, 128, 9) -> (N, 128*9)
    X_flat = X.reshape(X.shape[0], -1)

    # Convert to PyTorch Tensors
    dataset = TensorDataset(
        torch.tensor(X_flat, dtype=torch.float32),
        torch.tensor(y, dtype=torch.long)
    )
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

train_mask = np.isin(subjects_train, train_subs)
val_mask = np.isin(subjects_train, val_subs)
X_t, y_t = X_train[train_mask], y_train[train_mask]
X_v, y_v = X_train[val_mask], y_train[val_mask]

train_loader = create_dataloader(X_t, y_t, shuffle=True)
val_loader = create_dataloader(X_v, y_v)
test_loader = create_dataloader(X_test, y_test)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
INPUT_DIM = 128 * 9
HIDDEN_DIM = 256
NUM_CLASSES = 6

model = BaselineMLP(inputDim=INPUT_DIM, hiddenDim=HIDDEN_DIM, numClasses=NUM_CLASSES).to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss() # appropriate for multi class
optimizer = optim.Adam(model.parameters(), lr=0.001)

EPOCHS = 100

for epoch in range(EPOCHS):
    model.train()

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()           # Clear old gradients
        logits = model(X_batch)         # Forward pass
        loss = criterion(logits, y_batch) # Calculate loss
        loss.backward()                 # Backpropagation
        optimizer.step()                # Update weights


    # VALIDATION PHASE
    model.eval()
    val_preds, val_targets = [], []

    with torch.no_grad(): # Don't track gradients during validation (saves memory)
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(device)
            logits = model(X_batch)

            # Get the predicted class (highest probability)
            preds = torch.argmax(logits, dim=1)

            val_preds.extend(preds.cpu().numpy())
            val_targets.extend(y_batch.numpy())

    val_acc = accuracy_score(val_targets, val_preds)

    print(f"Epoch {epoch+1}/{EPOCHS} | Val Accuracy: {val_acc:.4f}")

Epoch 1/100 | Val Accuracy: 0.8717
Epoch 2/100 | Val Accuracy: 0.9212
Epoch 3/100 | Val Accuracy: 0.9355
Epoch 4/100 | Val Accuracy: 0.9362
Epoch 5/100 | Val Accuracy: 0.9416
Epoch 6/100 | Val Accuracy: 0.9443
Epoch 7/100 | Val Accuracy: 0.9443
Epoch 8/100 | Val Accuracy: 0.9525
Epoch 9/100 | Val Accuracy: 0.9470
Epoch 10/100 | Val Accuracy: 0.9477
Epoch 11/100 | Val Accuracy: 0.9464
Epoch 12/100 | Val Accuracy: 0.9491
Epoch 13/100 | Val Accuracy: 0.9464
Epoch 14/100 | Val Accuracy: 0.9477
Epoch 15/100 | Val Accuracy: 0.9416
Epoch 16/100 | Val Accuracy: 0.9443
Epoch 17/100 | Val Accuracy: 0.9423
Epoch 18/100 | Val Accuracy: 0.9443
Epoch 19/100 | Val Accuracy: 0.9362
Epoch 20/100 | Val Accuracy: 0.9498
Epoch 21/100 | Val Accuracy: 0.9504
Epoch 22/100 | Val Accuracy: 0.9464
Epoch 23/100 | Val Accuracy: 0.9538
Epoch 24/100 | Val Accuracy: 0.9484
Epoch 25/100 | Val Accuracy: 0.9511
Epoch 26/100 | Val Accuracy: 0.9484
Epoch 27/100 | Val Accuracy: 0.9498
Epoch 28/100 | Val Accuracy: 0.9437
E

In [ ]:
print("\nEvaluating on Test Set")
model.eval()
test_preds, test_targets = [], []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        logits = model(X_batch)
        preds = torch.argmax(logits, dim=1)

        test_preds.extend(preds.cpu().numpy())
        test_targets.extend(y_batch.numpy())

# Calculate final required metrics
final_acc = accuracy_score(test_targets, test_preds)
final_macro_f1 = f1_score(test_targets, test_preds, average='macro')

print(f"Final Test Accuracy: {final_acc:.4f}")
print(f"Final Test Macro-F1: {final_macro_f1:.4f}")